# 12. 차원 축소 (PCA, t-SNE)

## 학습 목표
- PCA(주성분 분석) 이해
- 차원 축소의 목적과 활용
- t-SNE를 이용한 시각화

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris, load_digits
import seaborn as sns

plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. PCA 기본 개념

In [ ]:
# Iris 데이터
iris = load_iris()
X = iris.data
y = iris.target

print(f"Original shape: {X.shape}")
print(f"Features: {iris.feature_names}")

In [ ]:
# Why: PCA finds directions of maximum variance, so features must be standardized first;
# otherwise high-magnitude features dominate the principal components regardless of
# their actual informational content.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA 적용
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print(f"Reduced shape: {X_pca.shape}")
print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total explained variance: {sum(pca.explained_variance_ratio_):.4f}")

In [ ]:
# PCA 결과 시각화
plt.figure(figsize=(10, 8))
colors = ['blue', 'green', 'red']

for i, (color, name) in enumerate(zip(colors, iris.target_names)):
    idx = y == i
    plt.scatter(X_pca[idx, 0], X_pca[idx, 1], c=color, label=name, 
                alpha=0.7, edgecolors='black')

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
plt.title('PCA - Iris Dataset')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 2. 설명 분산 분석

In [ ]:
# 전체 성분 PCA
pca_full = PCA()
pca_full.fit(X_scaled)

# Why: The cumulative explained variance curve shows how many components are needed
# to retain a target amount of information — the 95% threshold is a common heuristic
# that balances dimensionality reduction with minimal information loss.
cumsum = np.cumsum(pca_full.explained_variance_ratio_)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 개별 설명 분산
axes[0].bar(range(1, len(pca_full.explained_variance_ratio_)+1), 
            pca_full.explained_variance_ratio_)
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance Ratio')
axes[0].set_title('Explained Variance by Component')
axes[0].grid(True, alpha=0.3)

# 누적 설명 분산
axes[1].plot(range(1, len(cumsum)+1), cumsum, 'b-o')
axes[1].axhline(y=0.95, color='r', linestyle='--', label='95% threshold')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Explained Variance')
axes[1].set_title('Cumulative Explained Variance')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 주성분 해석

In [ ]:
# 주성분 로딩(가중치)
loadings = pd.DataFrame(
    pca_full.components_.T,
    columns=[f'PC{i+1}' for i in range(len(iris.feature_names))],
    index=iris.feature_names
)

print("=== Principal Component Loadings ===")
print(loadings.round(4))

In [ ]:
# 로딩 히트맵
plt.figure(figsize=(10, 6))
sns.heatmap(loadings.iloc[:, :2], annot=True, cmap='coolwarm', center=0)
plt.title('PCA Component Loadings')
plt.tight_layout()
plt.show()

## 4. t-SNE 시각화

In [ ]:
# Digits 데이터셋 (고차원)
digits = load_digits()
X_digits = digits.data
y_digits = digits.target

print(f"Digits shape: {X_digits.shape}")
print(f"Number of classes: {len(np.unique(y_digits))}")

In [ ]:
# PCA vs t-SNE 비교
# PCA
pca_digits = PCA(n_components=2)
X_pca_digits = pca_digits.fit_transform(X_digits)

# Why: t-SNE preserves local neighborhood structure by converting pairwise distances
# to conditional probabilities, making it far superior to PCA for visualizing clusters
# in high-dimensional data. perplexity=30 controls the effective neighborhood size.
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(X_digits)

print("Transformation complete!")

In [ ]:
# 비교 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# PCA
scatter1 = axes[0].scatter(X_pca_digits[:, 0], X_pca_digits[:, 1], 
                           c=y_digits, cmap='tab10', alpha=0.6, s=20)
axes[0].set_title('PCA - Digits Dataset')
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
plt.colorbar(scatter1, ax=axes[0], label='Digit')

# t-SNE
scatter2 = axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], 
                           c=y_digits, cmap='tab10', alpha=0.6, s=20)
axes[1].set_title('t-SNE - Digits Dataset')
axes[1].set_xlabel('t-SNE 1')
axes[1].set_ylabel('t-SNE 2')
plt.colorbar(scatter2, ax=axes[1], label='Digit')

plt.tight_layout()
plt.show()

## 5. PCA를 이용한 노이즈 제거

In [ ]:
# 노이즈 있는 Digits 이미지
np.random.seed(42)
noise = np.random.normal(0, 4, X_digits.shape)
X_noisy = X_digits + noise

# Why: Using only 20 principal components acts as a low-pass filter — noise lives in
# the low-variance components, so inverse_transform reconstructs the signal without
# the noise captured by the discarded components.
pca_denoise = PCA(n_components=20)
X_reconstructed = pca_denoise.inverse_transform(
    pca_denoise.fit_transform(X_noisy)
)

# 결과 시각화
fig, axes = plt.subplots(3, 10, figsize=(15, 5))

for i in range(10):
    # 원본
    axes[0, i].imshow(X_digits[i].reshape(8, 8), cmap='gray')
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_title('Original')
    
    # 노이즈
    axes[1, i].imshow(X_noisy[i].reshape(8, 8), cmap='gray')
    axes[1, i].axis('off')
    if i == 0:
        axes[1, i].set_title('Noisy')
    
    # 복원
    axes[2, i].imshow(X_reconstructed[i].reshape(8, 8), cmap='gray')
    axes[2, i].axis('off')
    if i == 0:
        axes[2, i].set_title('Denoised')

plt.suptitle('PCA Denoising (20 components)', fontsize=14)
plt.tight_layout()
plt.show()

## 정리

### PCA vs t-SNE

| 특성 | PCA | t-SNE |
|------|-----|-------|
| 목적 | 차원 축소, 분산 최대화 | 시각화, 군집 구조 |
| 선형성 | 선형 변환 | 비선형 변환 |
| 속도 | 빠름 | 느림 |
| 재현성 | 결정적 | random_state 필요 |
| 해석 | 가능 (로딩) | 어려움 |

### 활용
- **PCA**: 전처리, 노이즈 제거, 다중공선성 해소
- **t-SNE**: 고차원 데이터 시각화, 군집 탐색